In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "facebook/opt-125m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/251M [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [12]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
base = "facebook/opt-125m"  # 选个小模型走通流程；成功后再换 6.7B

tok = AutoTokenizer.from_pretrained(base)
model = AutoAWQForCausalLM.from_pretrained(base, device_map="auto")
print("Loaded base OK")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

OSError: Error no file named model.safetensors found in directory /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/27dcfa74d334bc871f3234de431e71c6eeba5dd6.

In [9]:
import torch
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

# ---------------------------
# 基本配置
# ---------------------------
base_model = "facebook/opt-6.7b"
quant_path = "models/opt-6.7b-awq"

# 建议：禁用grad & eval
torch.set_grad_enabled(False)

# ---------------------------
# Step 1: 准备 tokenizer
# ---------------------------
tokenizer = AutoTokenizer.from_pretrained(base_model)
# OPT 在批量生成时用左填充更稳（单条也可不设）
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ---------------------------
# Step 2: 加载原始模型（AWQ专用）
# ---------------------------
model = AutoAWQForCausalLM.from_pretrained(
    base_model,
    device_map="auto",          # 自动把层分配到可用 GPU
    torch_dtype=torch.float16,  # 半精度即可
)

# ---------------------------
# Step 3: 提供校准样本（最小示例，实际建议>128条短句）
# ---------------------------
# 真实项目里请采样你的目标域文本（游戏/客服/文档等）若干行，当作校准集
calib_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "In a distant city, players gather to challenge the arena champions.",
    "Merry Christmas! I'm glad to see everyone here today.",
    "The woman worked as a data scientist at a growing startup.",
    "Please ensure all requests are authenticated before processing.",
    "Defense switches to zone; the shooter relocates to the corner.",
] * 50  # 仅示例放大，实际可准备 200~1000 条

# ---------------------------
# Step 4: 量化配置 + 执行量化
# ---------------------------
quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM",  # 可选 GEMM / GEMV2
}

model.quantize(
    tokenizer,
    quant_config=quant_config,
    calib_data=calib_texts,     # 关键：提供校准数据
    max_calib_samples=512,      # 取前多少样本（可根据显存调整）
)

# ---------------------------
# Step 5: 保存量化模型
# ---------------------------
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

# ---------------------------
# Step 6: 用官方推荐 API 读取量化模型并测试推理
# ---------------------------
from awq import AutoAWQForCausalLM  # 重新引用，强调用 from_quantized
from transformers import TextStreamer

q_model = AutoAWQForCausalLM.from_quantized(
    quant_path,
    device_map="auto",
    fuse_layers=True,           # 通常能提速
    max_memory={0: "22GiB"}     # 多卡可传 dict；单卡可省略
)
q_model.eval()

def generate(text, max_new_tokens=64, temperature=0.8, top_p=0.95):
    inputs = tokenizer(text, return_tensors="pt").to(q_model.device)
    with torch.no_grad():
        out = q_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(generate("Merry Christmas! I'm glad to"))
print(generate("The woman worked as a"))


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

OSError: Error no file named model.safetensors found in directory /root/.cache/huggingface/hub/models--facebook--opt-6.7b/snapshots/a45aa65bbeb77c1558bc99bedc6779195462dab0.